In [44]:
import numpy as np
import pandas as pd
import re
from scipy import stats

## Data Overview

In [45]:
beer = pd.read_csv('github/data_cleaning/beers_info.csv')
beer.head()

,review_count,style_score,brewer,wgt_avg,beer_img,state,beer_rank,overall_score,beer_style,beer_name,abv,est_cal,mean,ibu,beer_desc
0,28,71.0,Fairhope Brewing Company,3.46,http://res.cloudinary.com/ratebeer/image/uploa...,ALABAMA,25,85.0,India Pale Ale (IPA),Fairhope (Take The) Causeway IPA,8.2%,246.0,3.54/5.0,69.0,Previously 6.5% abv and 75 IBUs
1,246,87.0,Anchorage Brewing Company,NaN,http://res.cloudinary.com/ratebeer/image/uploa...,ALASKA,25,97.0,Sour/Wild Ale,Anchorage Anadromous Belgian Black Bier,8.5%,255.0,/,30.0,Ale brewed with summit hops. Triple fermented....
2,9,NaN,McFate Brewing Company,3.89,http://res.cloudinary.com/ratebeer/image/uploa...,ARIZONA,25,99.0,Imperial Stout,McFate Imperial Candy Bar Stout - Barrel Aged,9.5%,285.0,4.29/5.0,NaN,NaN
3,15,48.0,Core Brewing and Distilling,3.13,http://res.cloudinary.com/ratebeer/image/uploa...,ARKANSAS,25,46.0,Brown Ale,Core Toasted Coconut Ale,6%,180.0,3.19/5.0,27.0,NaN
4,1130,100.0,Port Brewing Co / The Lost Abbey,NaN,http://res.cloudinary.com/ratebeer/image/uploa...,CALIFORNIA,25,100.0,American Strong Ale,Port Brewing Older Viscosity,12%,360.0,/,NaN,100% Bourbon Barrel Aged Strong Ale. This is ...


In [46]:
beer.shape

(1275, 15)

In [47]:
pd.DataFrame(beer.isnull().sum()).T   # number of missing values in each column

,review_count,style_score,brewer,wgt_avg,beer_img,state,beer_rank,overall_score,beer_style,beer_name,abv,est_cal,mean,ibu,beer_desc
0,0,45,0,742,0,0,0,8,0,0,646,429,0,678,125


In [48]:
map(lambda x: type(x), list(beer.iloc[0]))

[numpy.int64,
 numpy.float64,
 str,
 numpy.float64,
 str,
 str,
 numpy.int64,
 numpy.float64,
 str,
 str,
 str,
 numpy.float64,
 str,
 numpy.float64,
 str]

## Clean by Column

### 1. review_count (clean)

In [49]:
#beer.review_count.unique()   # review_count col is clean already

### 2. style_score

In [50]:
#list(beer.review_count[beer.style_score.isnull()])   # when missing style_score, review_count is low as well

In [51]:
#beer[beer.style_score.isnull()]   # run if you are interested

### 3. brewer

In [52]:
#beer.brewer.unique()

In [53]:
beer.brewer = map(lambda x: re.sub('\\xe2\\x80\\x99', '\'', x)\
                  .decode('utf-8', 'ignore').encode('ascii', 'ignore'), beer.brewer)

In [54]:
#beer.brewer.unique()

### 4. wgt_avg (clean)

In [55]:
#beer.wgt_avg.describe()  ### so many missing values, consider delete this column

### 5. beer_img (clean)

### 6. state (clean)

### 7. beer_rank (clean)

In [56]:
#beer.beer_rank.unique()

### 8. overall_score

In [57]:
#beer[beer.overall_score.isnull()]   # run if you are interested

### 9. beer_style

In [58]:
#beer.beer_style.unique()   # run if you are interested
#print type(beer['beer_style'][0])

In [59]:
beer['beer_style'] = ['Oktoberfest/Marzen' if 'Oktoberfest' in x else 'Kolsch' if 'lsch' in x else x for x in beer.beer_style]

In [60]:
beer['beer_style'] = map(lambda x: x.decode('utf-8', 'ignore').encode('ascii', 'ignore'), beer['beer_style'])

In [61]:
#beer['beer_style'].unique()

### 10. beer_name

In [62]:
a = map(lambda x: x.decode('utf-8').encode('ascii', 'ignore').replace('?', ''), beer.beer_name)
beer['beer_name'] = a

### 11. abv

In [ ]:
#beer.abv.describe()

In [ ]:
##aside
#a = np.nan
#print a
#print a == np.nan
#print a is np.nan

In [63]:
beer.abv = [str(x).replace("%", '') if x is not np.nan else np.nan for x in beer.abv]
beer.abv.describe()

count     629
unique    124
top        10
freq       36
Name: abv, dtype: object

### 12. est_cal (clean)

In [ ]:
#beer.est_cal.unique()

### 13. mean

In [ ]:
#beer['mean'].describe()
#beer['mean'].unique()

In [64]:
m1 = map(lambda x: re.sub('\/5.0', '', x), beer['mean'])
beer['mean'] = [np.nan if x == '/' else float(x) for x in m1]

In [68]:
#beer['mean']

### 14. ibu (clean)

In [ ]:
#beer['ibu'].describe()
#beer['ibu'].unique()
#type(beer['ibu'].unique()[0])

### 15. beer_desc

In [69]:
[bool(re.search('\\xe2\\x80\\x99', x)) if x is not np.nan else np.nan for x in beer['beer_desc']].count(True)

350

In [70]:
beer['beer_desc'] = [re.sub('\\xe2\\x80\\x99', '\'', x) if x is not np.nan else np.nan for x in beer['beer_desc']]

In [71]:
#[bool(re.search('\\xe2\\x80\\x99', x)) if x is not np.nan else np.nan for x in beer['beer_desc']].count(True) ## for test

0

In [73]:
beer['beer_desc'] = [x.decode('utf-8','ignore').encode('ascii', 'ignore') if x is not 
                     np.nan else np.nan for x in beer['beer_desc']]

## Arrange, Sort and Output

In [74]:
beer.columns
beer = beer[['state', 'beer_rank', 'beer_name', 'beer_style', 'brewer', 'review_count', 'overall_score',
'style_score', 'wgt_avg', 'abv', 'est_cal', 'mean', 'ibu', 'beer_desc', 'beer_img']]

In [75]:
beer.sort_values(['state', 'beer_rank'], ascending = [1, 1], inplace = True)
print beer.shape
beer.head()

(1275, 15)


,state,beer_rank,beer_name,beer_style,brewer,review_count,overall_score,style_score,wgt_avg,abv,est_cal,mean,ibu,beer_desc,beer_img
424,ALABAMA,1,Good People El Gordo Imperial Stout,Imperial Stout,Good People Brewing Company,15,99.0,90.0,3.89,13.9,417.0,4.13,65.0,NaN,http://res.cloudinary.com/ratebeer/image/uploa...
423,ALABAMA,2,Straight To Ale Laika Russian Imperial Stout -...,Imperial Stout,Straight To Ale,53,98.0,72.0,NaN,11.7,351.0,NaN,NaN,NaN,http://res.cloudinary.com/ratebeer/image/uploa...
422,ALABAMA,3,Straight To Ale Laika Russian Imperial Stout,Imperial Stout,Straight To Ale,55,97.0,71.0,NaN,NaN,74.0,NaN,NaN,"Second offering from the ""Right to Brew"" serie...",http://res.cloudinary.com/ratebeer/image/uploa...
421,ALABAMA,4,Avondale Barrel Aged Brett Saison,Saison,Avondale Brewing Company,10,97.0,95.0,3.69,7.5,225.0,4.01,NaN,This brew is our regular Saison aged in french...,http://res.cloudinary.com/ratebeer/image/uploa...
420,ALABAMA,5,Straight To Ale Laika Russian Imperial Stout -...,Imperial Stout,Straight To Ale,48,96.0,58.0,NaN,11.8,354.0,NaN,NaN,Straight To Ale Barrel Aged Laika Russian Impe...,http://res.cloudinary.com/ratebeer/image/uploa...


In [76]:
beer.to_csv('beer_info_proc.csv', index = False)